# Importing Libraries

In [1]:
import os
import re
import random
import csv
import chardet
import subprocess



import numpy as np
import pandas as pd
from numpy.linalg import norm

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from scipy.io import loadmat


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

import spacy
!python -m spacy download en_core_web_sm

from transformers import AutoTokenizer,AutoModel
from sentence_transformers import SentenceTransformer


import torch

from IPython.display import display, HTML

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Vishesh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vishesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 1.3 MB/s eta 0:00:10
     -- ------------------------------------- 0.8/12.8 MB 1.1 MB/s eta 0:00:11
     --- ------------------------------------ 1.0/12.8 MB 1.0 MB/s eta 0:00:12
     --- ------------------------------------ 1.0/12.8 MB 1.0 MB/s eta 0:00:12
     --- ----------------------------------- 1.3/12.8 MB 945.5 kB/s eta 0:00:13
     --- ----------------------------------- 1.3/12.8 MB 945.5 kB/s eta 0:00:13
     ---- ---------------------------------- 1.6/12.8 MB 830.6 kB/s eta 0:00:14
     ---- ---------------------------------- 1.6/12.8 MB 830.6 kB/s eta 0:00:14
     ----- --------------------------------- 1.8/12.8 MB 798.8 kB/s eta 0:00:14
     ----- --------------------------------- 1.8/12.8 MB 798.8 kB/s 

# Defining Embedding Methods

In [2]:
class Glove():
    def __init__(self,glove_file):
        self.glove_file=glove_file
        self.glove_dict={}

    def load_glove_model(self):
        with open(self.glove_file, 'r', encoding="utf-8") as f:
            for line in f:
                values = line.split()
                word = values[0]
                vector = np.array(values[1:], dtype=np.float32)
                self.glove_dict[word] = vector
    
    def get_glove_embeddings(self,words,embedding_dim=50):
        self.load_glove_model()
        embeddings = []
        for word in words:
            embeddings.append(self.glove_dict.get(word, np.zeros(embedding_dim)))  # Return zero vector if not found
        return embeddings


    def tokenize_sentence(self,sentence):
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(sentence)
        return [token.text.lower() for token in doc if token.is_alpha]  # Keep only words


    def sentence_to_glove_embedding(self,sentence):

        words = word_tokenize(sentence.lower())
        filtered_words = [word for word in words if word not in stopwords.words('english')]

        sentence=' '.join(filtered_words)

        tokens = self.tokenize_sentence(sentence)
        word_embeddings = self.get_glove_embeddings(tokens, 50)
        

        return word_embeddings,tokens
    
    def create_testfile_w_embedding(self,text_path,test_path,labels, texts):
        with open(text_path,"w") as file1,open(test_path,'w') as file2:
            row_wise_word_embedding=[]
            row_wise_tokens=[]
            for row in range(len(labels)):
                description=texts[row]
                text = ",".join(map(str, labels[row])) + " "
                text += description
                file1.write(text+"\n")

                word_embeddings,tokens=self.sentence_to_glove_embedding(description)
                row_wise_word_embedding.append(word_embeddings)
                row_wise_tokens.append(tokens)
                
                avg_pooled_embedding = np.mean(word_embeddings, axis=0, keepdims=True)

                text = ",".join(map(str, labels[row])) + " "
                text += " ".join(f"{i+1}:{100*value:.6f}" for i, value in enumerate(avg_pooled_embedding.flatten()))
                file2.write(text+"\n")

        return row_wise_word_embedding,row_wise_tokens

    def generate_colored_text(self, text, word_scores):
        word_scores = dict(word_scores)  # Convert word_scores to dictionary
        words = text.split()
        colored_words = []
        
        max_positive = max((score for score in word_scores.values() if score > 0), default=1)  # Avoid division by zero
        max_negative = max((-score for score in word_scores.values() if score < 0), default=1)  # Absolute max for negative

        for word in words:
            score = word_scores.get(word, 0)  # Default to 0 if word not in dictionary

            # Normalize blue opacity based on max positive value
            blue_opacity = max(0, min(score / max_positive, 1))  

            # Normalize red opacity based on max negative value
            red_opacity = max(0, min(-score / max_negative, 1))

            color = f"rgba(0, 0, 255, {blue_opacity})"
            if score < 0:
                color = f"rgba(255, 0, 0, {red_opacity})"

            colored_words.append(f'<span style="background-color: {color}; padding: 2px; border-radius: 3px; color: white;">{word}</span>')

        return " ".join(colored_words)

    def word_level_attribution(self,gradient_list_row,test_pred_path,row_embedding_list,row_tokens,classes,texts):
        row_wise_top_tokens_dict = []
     
        # for embedding_list, tokens in zip(row_embedding_list, row_tokens):
        with open(test_pred_path) as file:
            row=0
            top_tokens_per_class = {}
           
            for line in file:
                print(f"Text for row {row+1}:", end=" ")
                parts=line.split("\t")
                pred_labels = {int(k):float(v) for k, v in (item.split(":") for item in parts[1].split(','))}
                gradient_list=gradient_list_row[row]
                embedding_list=row_embedding_list[row]
                tokens=row_tokens[row]
                
                for i,key in enumerate(pred_labels):
                    print(f"Showing Integrated Gradients for {classes[key]} class")
                    
                    class_gradients=gradient_list[i]

                    attribution_scores = np.array([class_gradients[i] for i in range(min(50, len(class_gradients)))])
                    attribution_vector = attribution_scores.reshape(-1)  # Flatten
                    

                    token_importance = np.dot(embedding_list, attribution_vector) / (
                        np.linalg.norm(embedding_list, axis=1) * np.linalg.norm(attribution_vector) + 1e-10
                    )
                    

                    all_tokens = {tokens[i]: round(token_importance[i], 4) for i in range(len(tokens))}  # Store all tokens
                    sorted_tokens = dict(sorted(all_tokens.items(), key=lambda item: item[1], reverse=True)) 
                
                    top_tokens_per_class[classes[key]] = sorted_tokens
                    print(f"Attribution scores for all tokens in {classes[key]} class: {sorted_tokens}")
                    
                    html_output = self.generate_colored_text(texts[row], sorted_tokens)
                    display(HTML(f"<p>{html_output}</p>"))
                    


                row+=1
                row_wise_top_tokens_dict.append(top_tokens_per_class)
        
    

In [3]:
class SBert():
    def __init__(self,model,tokenizer):
        self.model=model
        self.tokenizer=tokenizer


    def backpropogate(self,sentence, gradients):
        gradients = torch.tensor(gradients, dtype=torch.float32).view(1, -1)
        tokens = self.tokenizer(sentence, return_tensors="pt", padding=False, truncation=True, return_attention_mask=True, add_special_tokens=False)
        
        # Extract input IDs & attention mask
        tokens_input_ids = tokens["input_ids"]
        tokens_attention_mask = tokens["attention_mask"]
        
        # Get token embeddings (leaf variable)
        with torch.no_grad():  # Prevent gradients from unnecessary parts
            tokens_input_embeds = self.model.embeddings.word_embeddings(tokens_input_ids)
        
        tokens_input_embeds.requires_grad_()  # Enable gradients on embeddings
        
        # Forward pass using token embeddings instead of input IDs
        output = self.model(inputs_embeds=tokens_input_embeds, attention_mask=tokens_attention_mask, output_hidden_states=True)
        
        # Obtain sentence embedding using mean pooling
        sentence_embedding = output.last_hidden_state.mean(dim=1)  # Mean pooling
        
        # Assume black-box model gradient wrt sentence embedding (obtained via finite differences)
        # Placeholder for actual gradients
        
        # Compute Jacobian of sentence embedding wrt token embeddings
        sentence_embedding.backward(gradients)
        
        # Get gradients of tokens
        token_gradients = tokens_input_embeds.grad  # Shape: (T, d)
        
        # Normalize for visualization
        token_gradients = np.array(token_gradients.norm(dim=-1)).reshape(-1)  # Get importance per token
        
        # Print token-wise gradient magnitudes
        decoded_tokens = self.tokenizer.convert_ids_to_tokens(tokens_input_ids.squeeze().tolist())
        return decoded_tokens, token_gradients


    def generate_colored_text(self, text, word_scores):
        word_scores = dict(word_scores)  # Convert word_scores to dictionary
        words = text.split()
        colored_words = []
        
        max_positive = max((score for score in word_scores.values() if score > 0), default=1)  # Avoid division by zero
        max_negative = max((-score for score in word_scores.values() if score < 0), default=1)  # Absolute max for negative

        for word in words:
            score = word_scores.get(word, 0)  # Default to 0 if word not in dictionary

            # Normalize blue opacity based on max positive value
            blue_opacity = max(0, min(score / max_positive, 1))  

            # Normalize red opacity based on max negative value
            red_opacity = max(0, min(-score / max_negative, 1))

            color = f"rgba(0, 0, 255, {blue_opacity})"
            if score < 0:
                color = f"rgba(255, 0, 0, {red_opacity})"

            colored_words.append(f'<span style="background-color: {color}; padding: 2px; border-radius: 3px; color: white;">{word}</span>')

        return " ".join(colored_words)
    

    def get_word_gradients(self,texts, test_pred_path, gradient_list_row,classes):
        with open(test_pred_path) as file:
            row=0
            counter=0
            gradient_list=gradient_list_row[row]
            
            for line in file:
                print(f"Integrated Gradients for test point {row+1}")
                print(f"Text for row {row+1}:", end=" ")
                print(texts[row])

                parts=line.split("\t")
                pred_labels = {int(k):float(v) for k, v in (item.split(":") for item in parts[1].split(','))}
                for key in pred_labels:
                    print(f"Showing Integrated Gradients for {classes[key]} class")
                    token_ids, backgradients = self.backpropogate(texts[row],gradient_list[counter])
                    # backgradients = np.mean(np.abs(backgradients),axis=2).reshape(-1)
                    backgradients_normalized = backgradients.copy()
                    backgradients_normalized -= np.min(backgradients_normalized)

                    max_val = np.max(backgradients_normalized)

                    if np.isnan(max_val) or max_val == 0:
                        backgradients_normalized = np.zeros_like(backgradients_normalized)  # Assign zeros or another fallback
                    else:
                        backgradients_normalized /= max_val

                    attributions = dict(zip(token_ids, backgradients))
                    attributions = sorted(attributions.items(), key=lambda item: item[1], reverse=True)
                    print(attributions)
                    
                    attributions = dict(zip(token_ids, backgradients_normalized))
                    attributions = sorted(attributions.items(), key=lambda item: item[1], reverse=True)
                    html_output = self.generate_colored_text(texts[row], attributions)
                    display(HTML(f"<p>{html_output}</p>"))
                    counter+=1
                    words = texts[row].split()
                    
                row+=1

    def create_testfile_w_embedding(self,text_path,test_path,labels, texts):
        with open(text_path,"w") as file1, open(test_path,"w") as file2:
            for row in range(len(labels)):
                description=texts[row]
                text = ",".join(map(str, labels[row])) + " "
                text += description
                file1.write(text+"\n")

                # print(description)
                tf_model=SentenceTransformer("all-MiniLM-L6-v2")
                embedding=tf_model.encode(description)
                embedding=embedding.tolist()

                text = ",".join(map(str, labels[row])) + " "
                text += " ".join(f"{i+1}:{100*value:.6f}" for i, value in enumerate(embedding))
                file2.write(text+"\n")

# Common Functions for both embedding methods

In [4]:
def load_class_words(filepath):
    with open(filepath, "r") as f:
        class_words = [line.strip() for line in f.readlines()]  # Remove spaces & newlines
    return class_words

def detect_encoding(file_path):
    with open(file_path, "rb") as f:
        raw_data = f.read(10000) 
    return chardet.detect(raw_data)["encoding"]

def extract_description(file_path):
    try:
        encoding = detect_encoding(file_path)  # Detect encoding
        with open(file_path, "r", encoding=encoding, errors="replace") as file:
            content = file.read()

        match = re.search(r"<DESCRIPTION>(.*?)</DESCRIPTION>", content, re.DOTALL)
        return match.group(1).strip() if match else None
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

In [5]:
def predict(test_path, result_file,trained_model_path):

    command = [
         os.path.join("AnnexML", "src", "annexml"), 
        "predict",
        os.path.join("AnnexML", "annexml-example.json"),
        f"predict_file={test_path}",
        f"model_file={trained_model_path}",
        f"result_file={result_file}"
    ]
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print("Standard Output:\n", result.stdout)
        if result.stderr:
            print("Standard Error:\n", result.stderr)
    except subprocess.CalledProcessError as e:
        pass
    print("Done.")

In [6]:
# Finding the intermediate samples
def generate_all_scales(test_path, scaled_test_path, step_size, num_steps):
    
    with open(test_path, 'r') as infile, open(scaled_test_path, 'w') as outfile:
        for line in infile:
            parts = line.split()
            labels = parts[0]
            values = [float(v) for k, v in (item.split(":") for item in parts[1:])]
        
            for i in range(0, num_steps):
                factor =  (i+1) / num_steps
                scaled_features = [v * factor for v in values]
                scaled_row = f"{labels} " + " ".join(f"{ind+1}:{v:.6f}" for ind,v in enumerate(scaled_features))
                outfile.write(scaled_row+"\n")

                for j in range(len(values)):
                    scaled_features = [v*factor for v in values]
                    scaled_features[j]+=step_size
                    scaled_row = f"{labels} " + " ".join(f"{ind+1}:{v:.6f}" for ind,v in enumerate(scaled_features))
                    outfile.write(scaled_row+"\n")

    print("done")

In [7]:
# function to calculate integrated gradients
def IG(test_path, test_pred_path, scaled_pred_path, step_size, num_steps,classes_labels):
    row = 1
    # class_feature_ig_dict = {}  # Dictionary to store IG values per class
    row_gradient_list = []

    with open(test_path, 'r') as infile1, open(test_pred_path, 'r') as infile2, open(scaled_pred_path, 'r') as infile3:
        gradient_list=[]
        for line1, line2 in zip(infile1, infile2):
            # print(f"Showing for {row}'th datapoint")

            parts = line1.split()
            true_labels = parts[0].split(",")
            
            values = np.array([float(v) for k, v in (item.split(":") for item in parts[1:])])  # feature values
            feature_indices = np.array([int(k) for k, v in (item.split(":") for item in parts[1:])])  # feature indices

            
            parts = line2.split("\t")
            pred_labels = {int(k):float(v) for k, v in (item.split(":") for item in parts[1].split(','))}
            
            for key in pred_labels:  # Iterate through each class
                # print(f"Showing Integrated Gradients for {classes_labels[key]}'th class")

                gradients = np.zeros(values.shape)  # Initialize gradients for each feature
                num_steps_arr = np.zeros(values.shape)
                
                basevalue=0
                infile3.seek(0)  # Reset the scaled predictions file
                for index, line3 in enumerate(infile3):
                    parts = line3.split("\t")
                    pred_labels_scaled = {int(k): float(v) for k, v in (item.split(":") for item in parts[1].split(','))}
                    
                    value = pred_labels_scaled.get(key, 0)  # Get the scaled prediction score for the class
        
                    
                    if index%(len(gradients)+1):
                        if value!=0 and basevalue!=0:
                            gradients[index%(len(gradients)+1)-1]+=value-basevalue
                            num_steps_arr[index%(len(gradients)+1)-1]+=1
                    else:
                         basevalue=value

                gradients /= step_size
                gradients /= num_steps_arr+1
                

                gradient_list.append(gradients)
               

                # print(gradients)
                            
            
            row_gradient_list.append(gradient_list)

                
            row += 1

    return np.array(row_gradient_list)







In [8]:
# to extract raw text from the annotation files and corresponding labels from the .mat file based on the indicies of lines to consider provided as list
def one_hot_to_indices(one_hot_labels):
    """Convert one-hot encoding to label indices."""
    return [list(np.where(row == 1)[0]) for row in one_hot_labels]
    


def get_test_data(lines_to_take):
    test_label_data = loadmat("IAPRTC/IAPRTC-12_TestLabels.mat")
    I_z_te=test_label_data['I_z_te']
    label_test_column_1 = I_z_te[0,0]
    test_labels = one_hot_to_indices(label_test_column_1)
    
    labels=[]
    texts = []
    with open("IAPRTC/iapr_test_list.txt") as file1:
        lines = file1.readlines() 
        for line_no in lines_to_take:
            if 1 <= line_no <= len(lines):  # Ensure the line number is valid
                path = "iaprtc12/annotations_complete_eng/"+lines[line_no-1][:-1]+".eng"
                description=extract_description(path)
                print(description)
                if not description=="":
                    labels.append(test_labels[line_no-1])
                    texts.append(description)
    
    return labels,texts


In [9]:
# classes from the dictionary
classes = []
with open("IAPRTC/iaprtc12_dictionary.txt") as file:
    for line in file:
        classes.append(line[:-1])
for i in range(len(classes)):  
    print(i, classes[i])

0 adult
1 airplane
2 airport
3 anorak
4 area
5 back
6 backpack
7 bag
8 balcony
9 bank
10 bar
11 base
12 bay
13 beach
14 bed
15 bedcover
16 bedside
17 bell
18 bench
19 bicycle
20 bike
21 bird
22 bit
23 blanket
24 bloom
25 board
26 boat
27 body
28 bone
29 bottle
30 boy
31 branch
32 brick
33 bridge
34 building
35 bus
36 bush
37 cactus
38 camera
39 canyon
40 cap
41 cape
42 car
43 carpet
44 cathedral
45 ceiling
46 centre
47 chair
48 child
49 church
50 city
51 classroom
52 cliff
53 clock
54 cloth
55 clothes
56 cloud
57 coast
58 cobblestone
59 column
60 condor
61 corner
62 corridor
63 couch
64 country
65 couple
66 court
67 courtyard
68 creek
69 cross
70 cup
71 curtain
72 cycling
73 cyclist
74 deck
75 desert
76 desk
77 dirt
78 dog
79 dome
80 door
81 dress
82 dune
83 edge
84 embankment
85 entrance
86 face
87 fence
88 fern
89 field
90 fjord
91 flag
92 flagpole
93 floor
94 flower
95 fog
96 footpath
97 forest
98 formation
99 fountain
100 frame
101 front
102 fruit
103 garden
104 gate
105 giant
106 

In [10]:
# to compute ig based on the two methods
def visualise(choice,lines,test_text_path,test_path,step_size,num_steps,test_pred_path,scaled_test_path,scaled_pred_path):


    class_words=load_class_words('IAPRTC/iaprtc12_dictionary.txt')


    test_pred_path = "annexml-result-temp.txt"
    scaled_test_path = "scaled_test.svm"
    scaled_pred_path = "annexml-result-temp_scaled.txt"


    labels, texts = get_test_data(lines)



    if choice == 1:
        tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
        model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

        trained_model_path='annexml-model-example.bin'

        sbert_embed=SBert(model,tokenizer)

        sbert_embed.create_testfile_w_embedding(test_text_path,test_path,labels,texts)

        predict(test_path,test_pred_path,trained_model_path)
        generate_all_scales(test_path, scaled_test_path, step_size, num_steps)
        predict(scaled_test_path,scaled_pred_path,trained_model_path)

        gradient_list = IG(test_path, test_pred_path, scaled_pred_path, step_size, num_steps,class_words)
        sbert_embed.get_word_gradients(texts=texts, test_pred_path=test_pred_path, gradient_list_row=gradient_list,classes=classes)

    elif choice == 2:
        glove_file = "glove.6B/glove.6B.50d.txt"

        glove_embed=Glove(glove_file)
        
        trained_model_path='annexml-model-example.bin'
        row_word_embedding,row_tokens=glove_embed.create_testfile_w_embedding(test_text_path,test_path,labels,texts)

        predict(test_path,test_pred_path,trained_model_path)
        generate_all_scales(test_path, scaled_test_path, step_size, num_steps)
        predict(scaled_test_path,scaled_pred_path,trained_model_path)


        gradient_list = IG(test_path, test_pred_path, scaled_pred_path, step_size, num_steps, class_words)

        
        glove_embed.word_level_attribution(gradient_list,test_pred_path,row_word_embedding,row_tokens,classes,texts)
    



In [ ]:
choice=1 #1 for sbert and 2 for glove 
lines=[1]
test_text_path = "temp_text_test.txt"
test_path = "temp_test.svm"
step_size=0.005
num_steps = 100
test_pred_path = "annexml-result-temp.txt"
scaled_test_path = "scaled_test.svm"
scaled_pred_path = "annexml-result-temp_scaled.txt"

visualise(choice,lines,test_text_path,test_path,step_size,num_steps,test_pred_path,scaled_test_path,scaled_pred_path)

a large building on the left, a palm tree in centre of picture, (mostly) white cars in the street at a junction, some of them turning left, others going straight; there are red umbrellas in a park on the right; people are walking through the park, others are crossing the road in the foreground;
Done.
done
Done.
Text for row 1: Showing Integrated Gradients for palm class
Attribution scores for all tokens in palm class: {'centre': 0.3353, 'street': 0.2442, 'building': 0.1885, 'park': 0.188, 'left': 0.1561, 'white': 0.1343, 'right': 0.0971, 'road': 0.087, 'turning': 0.0856, 'straight': 0.0692, 'others': 0.0509, 'mostly': 0.0449, 'red': 0.0343, 'walking': 0.0299, 'crossing': 0.0206, 'large': 0.0205, 'going': 0.0158, 'picture': 0.0127, 'umbrellas': 0.01, 'palm': -0.0114, 'people': -0.0124, 'junction': -0.0189, 'foreground': -0.062, 'cars': -0.0753, 'tree': -0.1809}


Showing Integrated Gradients for people class
Attribution scores for all tokens in people class: {'right': 0.2234, 'straight': 0.2175, 'white': 0.1841, 'turning': 0.1684, 'going': 0.1587, 'red': 0.1428, 'left': 0.1279, 'walking': 0.1147, 'people': 0.102, 'mostly': 0.085, 'others': 0.0758, 'umbrellas': 0.0652, 'park': 0.0629, 'palm': 0.0451, 'picture': 0.0356, 'building': 0.0344, 'street': 0.0168, 'large': 0.0121, 'tree': 0.0015, 'crossing': -0.0003, 'road': -0.003, 'junction': -0.0822, 'centre': -0.0836, 'foreground': -0.0982, 'cars': -0.1345}


Showing Integrated Gradients for sky class
Attribution scores for all tokens in sky class: {'centre': 0.2923, 'street': 0.258, 'building': 0.1925, 'park': 0.186, 'white': 0.1684, 'left': 0.1629, 'right': 0.1272, 'straight': 0.1131, 'turning': 0.1105, 'picture': 0.0684, 'red': 0.0663, 'road': 0.0651, 'others': 0.0635, 'mostly': 0.0422, 'going': 0.0377, 'umbrellas': 0.0289, 'large': 0.0235, 'walking': 0.0164, 'crossing': 0.0114, 'palm': -0.0023, 'people': -0.0074, 'foreground': -0.074, 'junction': -0.0785, 'cars': -0.0871, 'tree': -0.1891}


Showing Integrated Gradients for beach class
Attribution scores for all tokens in beach class: {'picture': 0.0982, 'street': 0.0864, 'people': 0.0855, 'left': 0.0829, 'red': 0.0703, 'large': 0.0661, 'umbrellas': 0.0604, 'palm': 0.0531, 'crossing': 0.0424, 'straight': 0.0416, 'going': 0.0354, 'tree': 0.0328, 'white': 0.0222, 'turning': 0.0183, 'others': 0.0121, 'foreground': -0.0036, 'right': -0.0066, 'walking': -0.0077, 'cars': -0.0131, 'mostly': -0.0191, 'road': -0.0495, 'park': -0.0523, 'junction': -0.102, 'building': -0.1249, 'centre': -0.1294}


Showing Integrated Gradients for tree class
Attribution scores for all tokens in tree class: {'junction': 0.3017, 'cars': 0.1704, 'road': 0.1222, 'tree': 0.072, 'foreground': 0.031, 'building': 0.0283, 'walking': 0.0264, 'crossing': 0.0111, 'park': 0.0024, 'mostly': 0.0015, 'centre': -0.0152, 'others': -0.0568, 'going': -0.0638, 'large': -0.0704, 'palm': -0.0712, 'turning': -0.0828, 'people': -0.0994, 'umbrellas': -0.1041, 'straight': -0.1117, 'right': -0.1183, 'left': -0.1197, 'red': -0.1368, 'picture': -0.1551, 'white': -0.1598, 'street': -0.1955}


Showing Integrated Gradients for square class
Attribution scores for all tokens in square class: {'centre': 0.3179, 'building': 0.1865, 'park': 0.1808, 'road': 0.1064, 'street': 0.0957, 'junction': 0.0808, 'left': 0.059, 'white': 0.0472, 'mostly': 0.046, 'right': 0.0416, 'walking': 0.0298, 'turning': 0.0272, 'others': 0.0271, 'crossing': 0.0026, 'straight': 0.0014, 'going': -0.0115, 'cars': -0.0306, 'large': -0.0316, 'umbrellas': -0.0426, 'foreground': -0.0474, 'red': -0.0498, 'palm': -0.0523, 'people': -0.06, 'picture': -0.0786, 'tree': -0.1613}


Showing Integrated Gradients for skyline class
Attribution scores for all tokens in skyline class: {'cars': 0.2088, 'junction': 0.1402, 'tree': 0.1215, 'foreground': 0.0543, 'umbrellas': 0.048, 'crossing': 0.0341, 'large': 0.0242, 'road': 0.024, 'people': 0.017, 'picture': -0.0057, 'palm': -0.0149, 'mostly': -0.0254, 'red': -0.0341, 'others': -0.0437, 'walking': -0.058, 'turning': -0.0691, 'left': -0.0821, 'going': -0.0871, 'building': -0.104, 'street': -0.1098, 'white': -0.1145, 'straight': -0.1293, 'park': -0.1493, 'right': -0.1714, 'centre': -0.2482}


Showing Integrated Gradients for city class
Attribution scores for all tokens in city class: {'cars': 0.2088, 'junction': 0.1402, 'tree': 0.1215, 'foreground': 0.0543, 'umbrellas': 0.048, 'crossing': 0.0341, 'large': 0.0242, 'road': 0.024, 'people': 0.017, 'picture': -0.0057, 'palm': -0.0149, 'mostly': -0.0254, 'red': -0.0341, 'others': -0.0437, 'walking': -0.058, 'turning': -0.0691, 'left': -0.0821, 'going': -0.0871, 'building': -0.104, 'street': -0.1098, 'white': -0.1145, 'straight': -0.1293, 'park': -0.1493, 'right': -0.1714, 'centre': -0.2482}


Showing Integrated Gradients for house class
Attribution scores for all tokens in house class: {'straight': 0.2322, 'white': 0.1288, 'right': 0.0599, 'picture': 0.0532, 'red': 0.025, 'going': -0.0027, 'umbrellas': -0.0093, 'turning': -0.0118, 'palm': -0.0148, 'road': -0.0182, 'street': -0.0189, 'centre': -0.0408, 'park': -0.0596, 'walking': -0.0607, 'mostly': -0.066, 'foreground': -0.0807, 'cars': -0.0849, 'left': -0.0961, 'junction': -0.0982, 'tree': -0.1186, 'crossing': -0.1502, 'others': -0.1549, 'people': -0.1606, 'building': -0.1774, 'large': -0.2159}


Showing Integrated Gradients for street class
Attribution scores for all tokens in street class: {'junction': 0.3263, 'centre': 0.1977, 'road': 0.1705, 'street': 0.1364, 'crossing': 0.1086, 'building': 0.1055, 'large': 0.0798, 'foreground': 0.0786, 'park': 0.0511, 'right': 0.0223, 'white': 0.0203, 'umbrellas': 0.0161, 'turning': 0.0148, 'tree': 0.0049, 'red': -0.0063, 'straight': -0.0354, 'left': -0.0473, 'walking': -0.0543, 'mostly': -0.0632, 'palm': -0.1166, 'others': -0.1173, 'picture': -0.1483, 'people': -0.1492, 'cars': -0.1533, 'going': -0.1821}
